In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import time

import missingno

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

In [2]:
df1 = pd.read_csv('투수_연봉_스탯_데이터_합본.csv')
df1

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,BK,WP,ERA,FIP,WHIP,WAR,팀,pid,연도,연봉등급
0,페냐,32,32,0,0,0,0,11,11,0,...,4,16,3.60,4.06,1.17,4.28,한화 이글스,15146,2023,A
1,문동주,23,23,0,0,0,0,8,8,0,...,1,5,3.72,3.65,1.31,3.08,한화 이글스,15013,2023,B
2,산체스,24,24,0,0,0,0,7,8,0,...,0,3,3.79,3.92,1.30,2.59,한화 이글스,15643,2023,A
3,주현상,55,0,55,9,0,0,2,2,0,...,0,0,1.96,3.25,0.84,2.44,한화 이글스,11415,2023,B
4,이태양,50,12,38,6,0,0,3,3,0,...,0,1,3.23,3.60,1.21,1.78,한화 이글스,10609,2023,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,박범준,5,0,5,3,0,0,0,1,0,...,0,2,13.50,5.59,2.06,-0.19,키움 히어로즈,16133,2024,D
468,전준표,18,5,13,4,0,0,2,4,0,...,0,4,6.83,7.09,1.99,-0.35,키움 히어로즈,16122,2024,D
469,오석주,17,0,17,7,0,0,1,1,0,...,0,2,11.12,4.40,2.29,-0.50,키움 히어로즈,12860,2024,D
470,조영건,25,3,22,3,0,0,2,1,0,...,0,3,8.01,6.01,1.91,-0.55,키움 히어로즈,14132,2024,D


In [5]:
df2 = pd.read_csv('23투수.csv')
df2

,Name_x,포지션,pit_G,GR,GF,CG,SHO,W,L,S,...,Ass,E,F%,RF9,수비 관련 득점 기여도.8,수비 관련 득점 기여도.9,연봉(만원),WAR,WAR당 연봉,팀_y
0,양현종,투수,29,0,0,0,0,9,11,0,...,0.0,0.0,0.0,0.0,0.0,0.0,50000,3.54,14142,KIA 타이거즈
1,진해수,투수,19,19,3,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,25000,-0.01,-3024251,LG 트윈스
2,고효준,투수,73,73,7,0,0,4,1,0,...,8.0,2.0,81.8,1.4,0.0,0.0,8500,0.78,10840,SSG 랜더스
3,김광현,투수,30,0,0,0,0,9,8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,100000,4.48,22341,SSG 랜더스
4,박종훈,투수,18,2,0,0,0,2,6,0,...,0.0,0.0,0.0,0.0,0.0,0.0,50000,0.39,129359,SSG 랜더스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,산체스,투수,24,0,0,0,0,7,8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,40000,2.51,15960,한화 이글스
293,엘리아스,투수,22,1,1,0,0,8,6,0,...,0.0,0.0,0.0,0.0,0.0,0.0,54000,3.54,15247,SSG 랜더스
294,맥키니,투수,12,0,0,0,0,1,9,0,...,0.0,0.0,0.0,0.0,0.0,0.0,10000,0.11,92461,키움 히어로즈
295,산체스,투수,12,1,1,1,0,4,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,25000,0.12,217065,KIA 타이거즈


In [9]:
df_position2023 = df2[['Name_x', '포지션', 'pid','연봉(만원)']]
df_position2023

,Name_x,포지션,pid,연봉(만원)
0,양현종,투수,10058,50000
1,진해수,투수,10075,25000
2,고효준,투수,10124,8500
3,김광현,투수,10126,100000
4,박종훈,투수,10131,50000
...,...,...,...,...
292,산체스,투수,15643,40000
293,엘리아스,투수,15644,54000
294,맥키니,투수,15646,10000
295,산체스,투수,15648,25000


In [11]:
df_position2023 = df_position2023.rename(columns={
    'Name_x': '선수'
})
df_position2023

,선수,포지션,pid,연봉(만원)
0,양현종,투수,10058,50000
1,진해수,투수,10075,25000
2,고효준,투수,10124,8500
3,김광현,투수,10126,100000
4,박종훈,투수,10131,50000
...,...,...,...,...
292,산체스,투수,15643,40000
293,엘리아스,투수,15644,54000
294,맥키니,투수,15646,10000
295,산체스,투수,15648,25000


In [13]:
# 연도가 2023 인 컬럼
df_2023 = df1[df1['연도'] == 2023]
df_2023

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,BK,WP,ERA,FIP,WHIP,WAR,팀,pid,연도,연봉등급
0,페냐,32,32,0,0,0,0,11,11,0,...,4,16,3.60,4.06,1.17,4.28,한화 이글스,15146,2023,A
1,문동주,23,23,0,0,0,0,8,8,0,...,1,5,3.72,3.65,1.31,3.08,한화 이글스,15013,2023,B
2,산체스,24,24,0,0,0,0,7,8,0,...,0,3,3.79,3.92,1.30,2.59,한화 이글스,15643,2023,A
3,주현상,55,0,55,9,0,0,2,2,0,...,0,0,1.96,3.25,0.84,2.44,한화 이글스,11415,2023,B
4,이태양,50,12,38,6,0,0,3,3,0,...,0,1,3.23,3.60,1.21,1.78,한화 이글스,10609,2023,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,김동규,3,1,2,0,0,0,0,1,0,...,0,4,22.50,13.95,3.75,-0.47,키움 히어로즈,15485,2023,D
227,김동혁,35,2,33,11,0,0,1,7,0,...,2,0,7.32,4.52,1.75,-0.51,키움 히어로즈,14674,2023,C
228,하영민,57,0,57,7,0,0,3,1,0,...,0,6,4.64,3.86,1.76,-0.70,키움 히어로즈,11222,2023,C
229,양현,54,0,54,12,0,0,0,5,0,...,1,0,5.05,5.28,1.82,-0.74,키움 히어로즈,10730,2023,C


In [19]:
# 두 컬럼 모두 기준으로 병합
df_2023 = df_2023.merge(df_position2023[['선수', '포지션', 'pid', '연봉(만원)']], 
                on=['선수', 'pid'], 
                how='left')
df_2023

,선수,G,GS,GR,GF,CG,SHO,W,L,S,...,ERA,FIP,WHIP,WAR,팀,pid,연도,연봉등급,포지션,연봉(만원)
0,페냐,32,32,0,0,0,0,11,11,0,...,3.60,4.06,1.17,4.28,한화 이글스,15146,2023,A,투수,55000
1,문동주,23,23,0,0,0,0,8,8,0,...,3.72,3.65,1.31,3.08,한화 이글스,15013,2023,B,투수,3300
2,산체스,24,24,0,0,0,0,7,8,0,...,3.79,3.92,1.30,2.59,한화 이글스,15643,2023,A,투수,40000
3,주현상,55,0,55,9,0,0,2,2,0,...,1.96,3.25,0.84,2.44,한화 이글스,11415,2023,B,투수,5800
4,이태양,50,12,38,6,0,0,3,3,0,...,3.23,3.60,1.21,1.78,한화 이글스,10609,2023,A,투수,66000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,김동규,3,1,2,0,0,0,0,1,0,...,22.50,13.95,3.75,-0.47,키움 히어로즈,15485,2023,D,투수,3000
249,김동혁,35,2,33,11,0,0,1,7,0,...,7.32,4.52,1.75,-0.51,키움 히어로즈,14674,2023,C,투수,6500
250,하영민,57,0,57,7,0,0,3,1,0,...,4.64,3.86,1.76,-0.70,키움 히어로즈,11222,2023,C,투수,6500
251,양현,54,0,54,12,0,0,0,5,0,...,5.05,5.28,1.82,-0.74,키움 히어로즈,10730,2023,C,투수,8500


In [21]:
df_2023.to_csv("2023년투수최종.csv",  index=False, encoding='utf-8-sig')

In [23]:
# CSV 파일 불러오기
batter_df = pd.read_csv("2023년_수비_스탯_연봉_데이터(포지션)최종.csv")
pitcher_df = pd.read_csv("2023년투수최종.csv")


# 필요한 컬럼만 추출
batter_df = batter_df[['선수', '팀', 'WAR', '연도', '포지션', '연봉(만원)']]
pitcher_df = pitcher_df[['선수', '팀', 'WAR', '연도', '포지션', '연봉(만원)']]

# 포지션구분 컬럼 추가
batter_df['포지션구분'] = '타자'
pitcher_df['포지션구분'] = '투수'

# 열 순서 맞추기
common_columns = ['선수', '팀', '연도', '포지션', '포지션구분', 'WAR', '연봉(만원)']
batter_df = batter_df[common_columns]
pitcher_df = pitcher_df[common_columns]

# 병합
merged_df = pd.concat([batter_df, pitcher_df], ignore_index=True)


In [25]:
merged_df

,선수,팀,연도,포지션,포지션구분,WAR,연봉(만원)
0,노시환,한화 이글스,2023,3루수,타자,6.53,13100.0
1,최재훈,한화 이글스,2023,포수,타자,3.23,80000.0
2,이진영,한화 이글스,2023,우익수,타자,2.44,3900.0
3,채은성,한화 이글스,2023,1루수,타자,1.79,180000.0
4,이도윤,한화 이글스,2023,유격수,타자,1.64,3400.0
...,...,...,...,...,...,...,...
540,김동규,키움 히어로즈,2023,투수,투수,-0.47,3000.0
541,김동혁,키움 히어로즈,2023,투수,투수,-0.51,6500.0
542,하영민,키움 히어로즈,2023,투수,투수,-0.70,6500.0
543,양현,키움 히어로즈,2023,투수,투수,-0.74,8500.0


In [27]:
merged_df.to_csv("2023_타자_투수_통합.csv", index=False)
